In [1]:
import torch
torch.cuda.empty_cache()
PYTORCH_NO_CUDA_MEMORY_CACHING=1
CUDA_LAUNCH_BLOCKING=1
# PYTORCH_CUDA_ALLOC_CONF = max_split_size_mb:<2>

In [2]:
l = "Learned counsel for the petitioner would submit that this matter is identical to the issue decided by this Court. This petition is may be disposed off in terms of the decision rendered by this court in bunch of the writ petitions in W.P.S. No. 209/2020 (Khomendra Kumar Ganjir v State of Chhattisgarh) and connected matters decided on 12.07.2022 wherein this Court has held as under :- “Learned counsel for the petitioners seek permission of this Court to withdraw the petitions with liberty to make representation in the light of the subsequent notification dated 02.05.2022 issued by the University Grant Commission wherein Dual Degree Programmae has been recognized. Relevant clause is extracted below:-  Dual Degree Programme 3.3.1. “Dual Degree Programme” shall be a programme jointly designed and offered by the Indian and Foreign Higher Educational Institutions in the same disciplines/subject areas and in the same level. The degrees for such programme shall be conferred by the Indian and Foreign Higher Educational Institutions, separately and simultaneously, upon completion of degree requirements of both the institutions. This shall not in any way be construed as two degree programmes in separate disciplines/subject areas and/or levels being pursued simultaneously. 3.3.2. Under the Dual degree programme, the degrees to be offered by an Indian Higher Educational Institution shall conform to the nomenclature and duration of the degrees as specified in section 22 (3) of the UGC Act, 1956 and shall also conform to minimum eligibility and other norms and standards to offer such degree programme. In view of the above submission, the petitions are dismissed with the aforesaid liberty in favour of the petitioners. It is directed that in the eventuality of filing of representation by the petitioners, the same shall be considered and decided by the respondent authorities within an outer limit of six weeks from the date of receipt of the representation. The interim relief passed earlier stands vacated cordingly, the writ petition is disposed off with the liberty granted in favour of the petitioner to file representation. It is directed that in the eventuality of filing of representation by the petitioner, the same shall be considered and decided by the respondent authorities within an outer limit of six weeks from the date of receipt of the representation."

In [3]:
def summary_gen(input_text):
#     !pip install --no-cache-dir transformers sentencepiece
    ARTICLE = input_text
    del input_text
    from transformers import pipeline
    import requests
    from transformers import AutoTokenizer
    tokenizer = AutoTokenizer.from_pretrained("nsi319/legal-pegasus",Use_fast=True)  
    max_chunk = 100
    summarizer = pipeline("summarization",model='nsi319/legal-pegasus',tokenizer=tokenizer,device=0)
    ARTICLE = ARTICLE.replace('.', '.<eos>')
    ARTICLE = ARTICLE.replace('?', '?<eos>')
    ARTICLE = ARTICLE.replace('!', '!<eos>')

    sentences = ARTICLE.split('<eos>')
    current_chunk = 0 
    chunks = []
    for sentence in sentences:
        if len(chunks) == current_chunk + 1: 
            if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
                chunks[current_chunk].extend(sentence.split(' '))
            else:
                current_chunk += 1
                chunks.append(sentence.split(' '))
        else:
            print(current_chunk)
            chunks.append(sentence.split(' '))

    for chunk_id in range(len(chunks)):
        chunks[chunk_id] = ' '.join(chunks[chunk_id])
  
    res = summarizer(chunks, max_length=128, min_length=30, do_sample=False)
  
    text = ' '.join([summ['summary_text'] for summ in res])
  
    return text

In [4]:
print(summary_gen(l))

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
text = ' '.join([summ['summary_text'] for summ in res])
# with open('/media/anubhav/New Volume/vidhaan/ai_models/judgement_summary/summary.txt', 'w') as f:
#     f.write(text)

In [ ]:
print(text)

In [ ]:
len(text)  #128


In [ ]:
len(text)  #256

In [ ]:
len(text)  #512

In [ ]:
512*6

In [ ]:
len(l)